In [2]:
!pip install -q pandas
!pip install -q numpy
!pip install -q scikit-learn
!pip install -q nltk

Created subset data matrix and a subset data recipe1,recipe2,similary formate datasets. 
used matrix only in next steps. 
similarity is calculated using TDIDF. 
further processing is done for thes matrix large_recipe_similarity_matrix in BertSimilarity_Matrix_subset_extraction_for_TDIDF_Analysis.ipynb / not further processing but creating a matched subset of bert similarties matrix

In [3]:
import pandas as pd
import pickle
import numpy as np
import nltk
#nltk.download('omw-1.4')


In [4]:

full_data=pd.read_csv('full_dataset.csv')
full_data['NER']
full_data.head()

,Unnamed: 0,title,ingredients,directions,link,source,NER
0,0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""peanut butter"", ""graham cracker crumbs"", ""bu..."


In [61]:
#backup
#full_data=data
data=full_data[:100000]

In [62]:
data['ingredients'] = data['ingredients'].apply(lambda x: x.replace("'", "").replace('"', '').replace("[", ""))

/var/folders/h7/3fnpld_x4n96hnvys2clxzcc0000gn/T/ipykernel_5965/2699800838.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['ingredients'] = data['ingredients'].apply(lambda x: x.replace("'", "").replace('"', '').replace("[", ""))


In [63]:
# # Count of unique ingredients per recipe
# data['ingredients'] = data['ingredients'].apply(lambda x: ','.join(x))

data['ingredients'].head()

0    1 c. firmly packed brown sugar, 1/2 c. evapora...
1    1 small jar chipped beef, cut up, 4 boned chic...
2    2 (16 oz.) pkg. frozen corn, 1 (8 oz.) pkg. cr...
3    1 large whole chicken, 2 (10 1/2 oz.) cans chi...
4    1 c. peanut butter, 3/4 c. graham cracker crum...
Name: ingredients, dtype: object

In [64]:
full_recipe_ingredients = dict(zip(data['title'], data['ingredients']))
full_recipe_ingredients

{'No-Bake Nut Cookies': '1 c. firmly packed brown sugar, 1/2 c. evaporated milk, 1/2 tsp. vanilla, 1/2 c. broken nuts (pecans), 2 Tbsp. butter or margarine, 3 1/2 c. bite size shredded rice biscuits]',
 "Jewell Ball'S Chicken": '1 small jar chipped beef, cut up, 4 boned chicken breasts, 1 can cream of mushroom soup, 1 carton sour cream]',
 'Creamy Corn': '2 cans whole kernel corn, drained, 1 block cream cheese, 1/2 small can mushroom pieces, drained]',
 'Chicken Funny': '1 large whole chicken, 2 (10 1/2 oz.) cans chicken gravy, 1 (10 1/2 oz.) can cream of mushroom soup, 1 (6 oz.) box Stove Top stuffing, 4 oz. shredded cheese]',
 'Reeses Cups(Candy)  ': '1 c. peanut butter, 3/4 c. graham cracker crumbs, 1 c. melted butter, 1 lb. (3 1/2 c.) powdered sugar, 1 large pkg. chocolate chips]',
 'Cheeseburger Potato Soup': '6 baking potatoes, 1 lb. of extra lean ground beef, 2/3 c. butter or margarine, 6 c. milk, 3/4 tsp. salt, 1/2 tsp. pepper, 1 1/2 c (6 oz.) shredded Cheddar cheese, divided, 

In [65]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')
# # Sample RecipeNLG dataset with recipe ingredients (for demonstration purposes)
# sample_recipe_ingredients = {
# 'Spaghetti Carbonara': 'spaghetti, eggs, pancetta, pecorino cheese, salt, black pepper',
# 'Chicken Curry': 'chicken, curry powder, garlic, onion, coconut milk, salt',
# 'Beef Stroganoff': 'beef, mushrooms, onion, garlic, sour cream, salt, black pepper',
# 'Vegetable Stir Fry': 'broccoli, bell peppers, carrots, garlic, soy sauce, salt',
# 'Fish Tacos': 'fish, tortillas, cabbage, lime, mayonnaise, salt',
# 'Pumpkin Soup': 'pumpkin, onion, garlic, vegetable broth, salt, black pepper',
# 'Chocolate Cake': 'flour, cocoa powder, sugar, eggs, butter, baking powder',
# 'Apple Pie': 'apples, flour, sugar, butter, cinnamon, salt',
# 'Caesar Salad': 'lettuce, croutons, parmesan cheese, caesar dressing, black pepper',
# 'Grilled Cheese': 'bread, cheese, butter, salt'
# }
# Display sample recipe ingredients
sample_recipe_ingredients=full_recipe_ingredients

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/divyamereddy/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [66]:
sample_recipe_ingredients

{'No-Bake Nut Cookies': '1 c. firmly packed brown sugar, 1/2 c. evaporated milk, 1/2 tsp. vanilla, 1/2 c. broken nuts (pecans), 2 Tbsp. butter or margarine, 3 1/2 c. bite size shredded rice biscuits]',
 "Jewell Ball'S Chicken": '1 small jar chipped beef, cut up, 4 boned chicken breasts, 1 can cream of mushroom soup, 1 carton sour cream]',
 'Creamy Corn': '2 cans whole kernel corn, drained, 1 block cream cheese, 1/2 small can mushroom pieces, drained]',
 'Chicken Funny': '1 large whole chicken, 2 (10 1/2 oz.) cans chicken gravy, 1 (10 1/2 oz.) can cream of mushroom soup, 1 (6 oz.) box Stove Top stuffing, 4 oz. shredded cheese]',
 'Reeses Cups(Candy)  ': '1 c. peanut butter, 3/4 c. graham cracker crumbs, 1 c. melted butter, 1 lb. (3 1/2 c.) powdered sugar, 1 large pkg. chocolate chips]',
 'Cheeseburger Potato Soup': '6 baking potatoes, 1 lb. of extra lean ground beef, 2/3 c. butter or margarine, 6 c. milk, 3/4 tsp. salt, 1/2 tsp. pepper, 1 1/2 c (6 oz.) shredded Cheddar cheese, divided, 

In [67]:
# Initialize Stemmer and Lemmatizer
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
# Function to preprocess, stem and lemmatize ingredients
def preprocess_ingredients(ingredients):
    # Tokenize and remove commas
    tokens = ingredients.replace(',', '').split()
    # Apply stemming
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    # Apply lemmatization
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in stemmed_tokens]
    return ' '.join(lemmatized_tokens)
# Apply preprocessing, stemming and lemmatization to the sample ingredients
processed_ingredients = {title: preprocess_ingredients(ingredients) for title, ingredients in sample_recipe_ingredients.items()}
# Display processed ingredients
processed_ingredients

{'No-Bake Nut Cookies': '1 c. firmli pack brown sugar 1/2 c. evapor milk 1/2 tsp. vanilla 1/2 c. broken nut (pecans) 2 tbsp. butter or margarin 3 1/2 c. bite size shred rice biscuits]',
 "Jewell Ball'S Chicken": '1 small jar chip beef cut up 4 bone chicken breast 1 can cream of mushroom soup 1 carton sour cream]',
 'Creamy Corn': '2 can whole kernel corn drain 1 block cream chees 1/2 small can mushroom piec drained]',
 'Chicken Funny': '1 larg whole chicken 2 (10 1/2 oz.) can chicken gravi 1 (10 1/2 oz.) can cream of mushroom soup 1 (6 oz.) box stove top stuf 4 oz. shred cheese]',
 'Reeses Cups(Candy)  ': '1 c. peanut butter 3/4 c. graham cracker crumb 1 c. melt butter 1 lb. (3 1/2 c.) powder sugar 1 larg pkg. chocol chips]',
 'Cheeseburger Potato Soup': '6 bake potato 1 lb. of extra lean ground beef 2/3 c. butter or margarin 6 c. milk 3/4 tsp. salt 1/2 tsp. pepper 1 1/2 c (6 oz.) shred cheddar chees divid 12 slice bacon cook crumbl and divid 4 green onion chop and divid 1 (8 oz.) cart

In [68]:
# Initialize TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()
# Fit and transform the processed ingredients to get the TF-IDF matrix
tfidf_matrix = tfidf_vectorizer.fit_transform(list(processed_ingredients.values()))
# Calculate cosine similarity between the TF-IDF vectors
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
# Create a DataFrame to store the similarity values
similarity_df = pd.DataFrame(cosine_sim, index=processed_ingredients.keys(), columns=processed_ingredients.keys())
# Display the similarity DataFrame
similarity_df

,No-Bake Nut Cookies,Jewell Ball'S Chicken,Creamy Corn,Chicken Funny,Reeses Cups(Candy),Cheeseburger Potato Soup,Rhubarb Coffee Cake,Scalloped Corn,Nolan'S Pepper Steak,Millionaire Pie,...,Parmesan Pillows,Potato Rags,Blueberry Sponge Cake Roll,Hot German Bean Salad,Butternut Souffle,Creamy Orange Tapioca,Wet And Good Chocolate Cake,French Toast(Low Cholesterol),Black Bean Soup-Cuban Style(Frijoles Negros),Holiday Trash
No-Bake Nut Cookies,1.000000,0.000000,0.000000,0.032241,0.055367,0.089587,0.076351,0.049153,0.009071,0.018532,...,0.039924,0.000000,0.072422,0.023363,0.125234,0.191530,0.144874,0.107343,0.023429,0.117439
Jewell Ball'S Chicken,0.000000,1.000000,0.180472,0.220109,0.000000,0.176895,0.000000,0.016862,0.158694,0.075244,...,0.000000,0.000000,0.055249,0.104596,0.025939,0.000000,0.073907,0.030506,0.047782,0.042096
Creamy Corn,0.000000,0.180472,1.000000,0.163112,0.000000,0.031967,0.000000,0.318406,0.068739,0.114189,...,0.000000,0.227764,0.017177,0.115407,0.054979,0.000000,0.000000,0.000000,0.048139,0.064539
Chicken Funny,0.032241,0.220109,0.163112,1.000000,0.032480,0.110258,0.000000,0.143804,0.020450,0.153878,...,0.065010,0.000000,0.029598,0.143641,0.039331,0.000000,0.074714,0.048417,0.068877,0.046796
Reeses Cups(Candy),0.055367,0.000000,0.000000,0.032480,1.000000,0.045580,0.016250,0.130763,0.019347,0.170399,...,0.082365,0.000000,0.021147,0.026845,0.000000,0.019703,0.161321,0.038990,0.065170,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Creamy Orange Tapioca,0.191530,0.000000,0.000000,0.000000,0.019703,0.033016,0.017645,0.016884,0.013505,0.046561,...,0.000000,0.000000,0.222727,0.023410,0.163160,1.000000,0.135687,0.015490,0.000000,0.000000
Wet And Good Chocolate Cake,0.144874,0.073907,0.000000,0.074714,0.161321,0.117511,0.254740,0.030537,0.030216,0.028920,...,0.045786,0.000000,0.160273,0.144811,0.220984,0.135687,1.000000,0.086628,0.085993,0.119211
French Toast(Low Cholesterol),0.107343,0.030506,0.000000,0.048417,0.038990,0.060590,0.131594,0.049705,0.009683,0.019783,...,0.000000,0.000000,0.075037,0.060870,0.089992,0.015490,0.086628,1.000000,0.065374,0.071799
Black Bean Soup-Cuban Style(Frijoles Negros),0.023429,0.047782,0.048139,0.068877,0.065170,0.115907,0.089628,0.027552,0.090652,0.000000,...,0.000000,0.030630,0.042356,0.150045,0.050767,0.000000,0.085993,0.065374,1.000000,0.042103


In [69]:
# Save the expanded similarity matrix to a CSV file
large_csv_file_path = 'large_recipe_similarity_matrix.csv'
similarity_df.to_csv(large_csv_file_path)
large_csv_file_path

'large_recipe_similarity_matrix.csv'

In [6]:
# large_csv_file_path = 'Data/large_recipe_similarity_matrix.csv'
# similarity_df=pd.read_csv(large_csv_file_path)

In [ ]:
similarity_df

,No-Bake Nut Cookies,Jewell Ball'S Chicken,Creamy Corn,Chicken Funny,Reeses Cups(Candy),Cheeseburger Potato Soup,Rhubarb Coffee Cake,Scalloped Corn,Nolan'S Pepper Steak,Millionaire Pie,...,Parmesan Pillows,Potato Rags,Blueberry Sponge Cake Roll,Hot German Bean Salad,Butternut Souffle,Creamy Orange Tapioca,Wet And Good Chocolate Cake,French Toast(Low Cholesterol),Black Bean Soup-Cuban Style(Frijoles Negros),Holiday Trash
No-Bake Nut Cookies,1.000000,0.000000,0.000000,0.032241,0.055367,0.089587,0.076351,0.049153,0.009071,0.018532,...,0.039924,0.000000,0.072422,0.023363,0.125234,0.191530,0.144874,0.107343,0.023429,0.117439
Jewell Ball'S Chicken,0.000000,1.000000,0.180472,0.220109,0.000000,0.176895,0.000000,0.016862,0.158694,0.075244,...,0.000000,0.000000,0.055249,0.104596,0.025939,0.000000,0.073907,0.030506,0.047782,0.042096
Creamy Corn,0.000000,0.180472,1.000000,0.163112,0.000000,0.031967,0.000000,0.318406,0.068739,0.114189,...,0.000000,0.227764,0.017177,0.115407,0.054979,0.000000,0.000000,0.000000,0.048139,0.064539
Chicken Funny,0.032241,0.220109,0.163112,1.000000,0.032480,0.110258,0.000000,0.143804,0.020450,0.153878,...,0.065010,0.000000,0.029598,0.143641,0.039331,0.000000,0.074714,0.048417,0.068877,0.046796
Reeses Cups(Candy),0.055367,0.000000,0.000000,0.032480,1.000000,0.045580,0.016250,0.130763,0.019347,0.170399,...,0.082365,0.000000,0.021147,0.026845,0.000000,0.019703,0.161321,0.038990,0.065170,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Creamy Orange Tapioca,0.191530,0.000000,0.000000,0.000000,0.019703,0.033016,0.017645,0.016884,0.013505,0.046561,...,0.000000,0.000000,0.222727,0.023410,0.163160,1.000000,0.135687,0.015490,0.000000,0.000000
Wet And Good Chocolate Cake,0.144874,0.073907,0.000000,0.074714,0.161321,0.117511,0.254740,0.030537,0.030216,0.028920,...,0.045786,0.000000,0.160273,0.144811,0.220984,0.135687,1.000000,0.086628,0.085993,0.119211
French Toast(Low Cholesterol),0.107343,0.030506,0.000000,0.048417,0.038990,0.060590,0.131594,0.049705,0.009683,0.019783,...,0.000000,0.000000,0.075037,0.060870,0.089992,0.015490,0.086628,1.000000,0.065374,0.071799
Black Bean Soup-Cuban Style(Frijoles Negros),0.023429,0.047782,0.048139,0.068877,0.065170,0.115907,0.089628,0.027552,0.090652,0.000000,...,0.000000,0.030630,0.042356,0.150045,0.050767,0.000000,0.085993,0.065374,1.000000,0.042103


In [72]:
# # Reset index for melting


# # Rename the index column if it doesn't automatically get named 'index'
# similarity_df.rename(columns={'index': 'Recipe1'}, inplace=True)

# # Melt the DataFrame to long format
# melted_df = pd.melt(similarity_df, id_vars=['Recipe1'], value_vars=processed_ingredients.keys())

# # Rename columns
# melted_df.columns = ['Recipe1', 'Recipe1', 'similarity']

# # Display the melted DataFrame
# print(melted_df)

import pandas as pd

# Stack the DataFrame and reset the index
stacked_df = similarity_df.stack().reset_index()

# Rename the columns
stacked_df.columns = stacked_df.columns

# # Optionally, remove self-similarity entries (where item1 == item2)
# stacked_df = stacked_df[stacked_df['item1'] != stacked_df['item2']]

# Display the long-format DataFrame
print(stacked_df)

: 

In [3]:
# import pandas as pd
# stacked_df=pd.read_csv('Data/large_stacked_df_TF_IDF_long_data.csv')

In [58]:
stacked_df.head()

,level_0,level_1,0
0,No-Bake Nut Cookies,No-Bake Nut Cookies,1.000000
1,No-Bake Nut Cookies,Jewell Ball'S Chicken,0.000000
2,No-Bake Nut Cookies,Creamy Corn,0.068567
3,No-Bake Nut Cookies,Chicken Funny,0.033579
4,No-Bake Nut Cookies,Reeses Cups(Candy),0.073766
...,...,...,...
9995,Brickle Bars,Baked Beans,0.014128
9996,Brickle Bars,Sweet-N-Sour Chicken,0.000000
9997,Brickle Bars,Ambrosia,0.062505
9998,Brickle Bars,Crazy Peanut Butter Cookies,0.062725


In [ ]:
stacked_df.to_csv('Data/large_stacked_df_TF_IDF_long_data.csv')